In [ ]:
# Colab only: Run this cell to download/install packages
import sysu

# Graph Library


In this problem, you're going to write a (very minimal) graph library, which uses both the adjacency dictionary and the sparse adjacency matrix representation of a graph.  Using these two representations, you'll implement two of the more well-known large-scale graph processing algorithms: Dijkstra's algorithm for finding single-source shortest paths in the graph, and the PageRank algorithm for determining the importance of nodes in the network.

### The `wikipedia_small` graph

The graph we'll be focusing on for most of this assignment is a directed graph that represents the page links in the English language Wikipedia.  Specifically, we took the (pre-processed) Wikipedia dump from `http://haselgrove.id.au/wikipedia.htm`, which were taken from a 2008 version of Wikipedia, and we then subselected only those nodes that had at least _500 incoming links_.  This resulted in a graph with about 24,000 nodes and 6,000,000 edges.

*Note*: The `http://haselgrove.id.au/wikipedia.htm` has since died off. See Wikipedia itself for information on accessing data: https://en.wikipedia.org/wiki/Wikipedia:Database_download#Where_do_I_get_it?

There are two files included with this notebook that are relevant here:

- `wikipedia_small.graph.gz`
- `wikipedia_small.nodes.gz`
    
The `.graph.gz` file contains a (gzipped) list of integers, two per line.  If the line "`i j`" appears in the file, this indicates a directed edge from node `i` to node `j`.  The `.nodes.gz` file then contains a (gzipped) list of each node the graph, where the link number indicates the node index.  This is how we can relate the node numbers in the `.graph.gz` file to actual pages on Wikipedia.

## Your own Graph class

In the main portion of this assignment, you'll create your own Graph class that mimics some of the functionality of networkx. We'll provide you with some scaffolding and support code.

In [1]:
import numpy as np
import scipy.sparse as sp
import heapdict
import heapq
import gzip
import networkx as nx
from math import inf

# Utility function to read the graph
def read_graph(basename="wikipedia_small"):
    with gzip.open(f"D:\Study\Data Science\Lab2\wikipedia_small.nodes.gz", 'rt', encoding="utf-8") as f:
        nodes = [a.strip() for a in f]
    with gzip.open(f"D:\Study\Data Science\Lab2\wikipedia_small.graph.gz", 'rt', encoding="utf-8") as f:
        links = []
        for row in f:
            i, j = tuple(row.strip().split())
            links.append((nodes[int(i)], nodes[int(j)]))
    return links

### Q1: Implement `add_edges`

Here is the template for your Graph class.

Note that `self.edges` should be represented as an "adjacency dictionary", so that for every node `i` in the graph `self.edges[i]` is a dictionary of nodes that `i` is directly connected to. The value of the inner dictionary should be `1` for every edge that exists (the value of this entry doesn't matter, so we could technically use a dictionary of sets, but we use a dictionary of dictionaries to keep things a little bit more uniform and to allow for potential extensions e.g. to weighted graphs.)

Note that all vertices must exist in the dictionary. If a vertex has no outgoing edges, then it should still have an entry pointing to an empty dictionary. 

To begin, implement the function `add_edges()`. It must modify the `self.edges` variable to add all edges passed as tuples in `edges_list`.

In [7]:
from heapdict import heapdict

class Graph:
    def __init__(self):
        self.edges = {}

    def add_edges(self, edges_list):
        for edge in edges_list:
            # Ensure vertices exist in the dictionary
            self.edges.setdefault(edge[0], {})
            self.edges.setdefault(edge[1], {})
            
            # Add edges to the adjacency dictionary
            self.edges[edge[0]][edge[1]] = 1
            self.edges[edge[1]][edge[0]] = 1  # Assuming an undirected graph
    def adjacency_matrix(self):
        """Compute an adjacency matrix form of the graph.
        
        Returns: tuple (A, nodes)
            A: a sparse matrix in COO form that represents the adjacency matrix
               for the graph (i.e., A[j, i] = 1 iff there is an edge i->j)
               NOTE: be sure you have this ordering correct!
            nodes: a list of nodes indicating the node key corresponding to each
               index of the A matrix
        """
        # Get the unique nodes in the graph
        nodes = list(self.edges.keys())
        
        # Create dictionaries to map nodes to indices and vice versa
        node_to_index = {node: index for index, node in enumerate(nodes)}
        index_to_node = {index: node for index, node in enumerate(nodes)}
        
        # Initialize lists to store COO matrix data
        row_indices = []
        col_indices = []
        data = []
        
        # Populate COO matrix data
        for i, node in enumerate(nodes):
            for neighbor in self.edges[node]:
                row_indices.append(node_to_index[node])
                col_indices.append(node_to_index[neighbor])
                data.append(1)
        
        # Create the COO matrix
        A = sp.coo_matrix((data, (row_indices, col_indices)), shape=(len(nodes), len(nodes)))
        
        return A, nodes
    def pagerank(self, d, iters):
        """Compute the PageRank score for each node in the network.

        Args:
            d: PageRank damping factor d (restart with probability (1-d))
            iters: number of iterations to run

        Returns:
            dict ranks: a dictionary of node: importance score, for each node in the
                network (larger score means higher rank)
        """
        # Get the adjacency matrix and nodes
        adj_matrix, nodes = self.adjacency_matrix()

        # Normalize adjacency matrix to get the transition matrix P
        P = adj_matrix / adj_matrix.sum(axis=0)

        # Get the number of nodes
        n = len(nodes)

        # Initialize the probability vector x with uniform distribution
        x = np.ones(n) / n

        # Power iteration method for PageRank
        for _ in range(iters):
            x = d * P.dot(x) + (1 - d) / n

        # Create a dictionary of nodes and their corresponding PageRank scores
        ranks = {nodes[i]: x[i] for i in range(n)}
        sorted_ranks = sorted(ranks.items(), key=lambda x: x[1], reverse=True)

        return dict(sorted_ranks)

        
my_graph = Graph()
my_graph.add_edges([('A', 'B'), ('A', 'C'), ('B', 'D')])

print(my_graph.edges)




{'A': {'B': 1, 'C': 1}, 'B': {'A': 1, 'D': 1}, 'C': {'A': 1}, 'D': {'B': 1}}


## Q2: Dijkstra's algorithm

Next, implement Dijkstra's single-source shortest path algorithm (with the simple case where the distance along any edge is assumed to be one).  You can refer to the [Wikipedia page on Dijkstra's algorithm](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm) for reference. The basic idea of the algorithm is to keep a priority queue of nodes ordered by distance from a source node.  At each step, we continually pop off the smallest element `i` in the queue, and update the distance of all successor nodes to have a distance of `1 + distance[i]`.

For the priority queue, you should use a [`heapdict`](https://github.com/DanielStutzbach/heapdict), which is a form of priority queue that allows you to change the priority of an element.

When called with source node `A`, the function should return a dictionary where the keys are all the nodes in the graph. For each key `B`:

- if `B` is reachable from `A` then the value must be the tuple `(distance, prev_node)`, where:
  - `distance` is the minimum number of hops from `A` to `B`, and
  - `prev_node` is the node immediately before `B` along one such shortest path
- if `B` is not reachable from `A`, then the value must be `(inf,None)`
- if `B == A`, then the value should be `(0, None)`

In [6]:
def shortest_path(g, source):
    """ Compute the single-source shorting path.

    This function uses Dijkstra's algorithm to compute the distance from 
    source to all other nodes in the network.

    Args:
        g: Graph object
        source: source node

    Returns:
        dictionary of node: (distance, prev_node) values for each reachable node in the graph, where 
              distance denotes the shortest distance from of node from source, and
              prev_node is the previous node on the shortest path from source to node.
            if node is unreachable, the value should be None
    """
    distances = {vertex: float('infinity') for vertex in g}
    distances[source] = 0

    pq = [(0, source)]
    while len(pq) > 0:
            current_distance, current_source = heapq.heappop(pq)

            if current_distance > distances[current_source]:
                continue

            for neighbor, weight in g[current_source].items():
                distance = current_distance + weight

                if distance < distances[neighbor]:
                    distances[neighbor] = distance
                    heapq.heappush(pq, (distance, neighbor))

    return distances


example_graph = {
    'A': {'B': 1, 'C': 1, 'D': 1},
    'B': {'C': 1, 'D': 1},
    'C': {'A': 1, 'D': 1},
    'D': {'B': 1, 'C': 1}
}
# my_graph = Graph()
# example_graph = my_graph.edges
print (example_graph)
print(shortest_path(example_graph, 'B'))

{'A': {'B': 1, 'C': 1, 'D': 1}, 'B': {'C': 1, 'D': 1}, 'C': {'A': 1, 'D': 1}, 'D': {'B': 1, 'C': 1}}
{'A': 2, 'B': 0, 'C': 1, 'D': 1}


## Q3: Adjacency matrix representation

Implement the adjacency matrix method of the Graph class.  This returns a matrix representing the adjacency of the graph (in scipy COO sparse format), as well as a list of nodes that indicate how the indices in this graph relate to the nodes in the network.

In order to complete this question in a manner that works on the Wikipedia graph, you must implement this function natively as a sparse matrix (i.e., you cannot construct a dense matrix and then convert that to a sparse matrix, but need to directly use the `sp.coo_matrix()` constructor).  The Wikipedia graph is 24K x 24K nodes, which (assuming 8 bytes per entry, would take up 4GB of memory.  While it's not impossible to do things this way at this scale (it quickly becomes infeasible for graphs that are even slightly larger), it's a very bad idea, and just allocating this much memory will take too long.

**Note the order of the axes of the output matrix.** This is important for calculating the PageRank.

In [6]:
def adjacency_matrix(self):
        """Compute an adjacency matrix form of the graph.
        
        Returns: tuple (A, nodes)
            A: a sparse matrix in COO form that represents the adjacency matrix
               for the graph (i.e., A[j, i] = 1 iff there is an edge i->j)
               NOTE: be sure you have this ordering correct!
            nodes: a list of nodes indicating the node key corresponding to each
               index of the A matrix
        """
        # Get the unique nodes in the graph
        nodes = list(self.edges.keys())
        
        # Create dictionaries to map nodes to indices and vice versa
        node_to_index = {node: index for index, node in enumerate(nodes)}
        index_to_node = {index: node for index, node in enumerate(nodes)}
        
        # Initialize lists to store COO matrix data
        row_indices = []
        col_indices = []
        data = []
        
        
        for i, node in enumerate(nodes):
            for neighbor in self.edges[node]:
                row_indices.append(node_to_index[node])
                col_indices.append(node_to_index[neighbor])
                data.append(1)
        
        # Create the COO matrix
        A = sp.coo_matrix((data, (row_indices, col_indices)), shape=(len(nodes), len(nodes)))
        
        return A, nodes

# Example usage:

graph = Graph()


edges_list = read_graph()
graph.add_edges(edges_list)

# Get the adjacency matrix and nodes
adj_matrix, nodes = graph.adjacency_matrix()

# Print or use the adjacency matrix and nodes as needed
print("Adjacency Matrix:")
print(adj_matrix.toarray())  # Convert to dense array for printing

print("\nNodes:")
print(nodes)


Adjacency Matrix:
[[0 1 1 ... 0 0 0]
 [1 0 1 ... 0 0 1]
 [1 1 0 ... 0 0 1]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 1 ... 0 0 0]]

Nodes:
['.NET_Framework', '2004', '2005', '2006', '2007', '2008', '3D_computer_graphics', 'API', 'Algorithm', 'Application_programming_interface', 'April_16', 'C++', 'C_Sharp_(programming_language)', 'Compiler', 'Computer_networking', 'Cross-platform', 'Cryptography', 'Database', 'December_1', 'December_20', 'DirectX', 'Distributed_computing', 'FreeBSD', 'Free_software', 'GPL', 'Hewlett-Packard', 'Intel', 'International_Organization_for_Standardization', 'January_22', 'January_25', 'Java_(programming_language)', 'July_3', 'July_7', 'June_1', 'June_2', 'June_26', 'List_of_software_categories', 'Mac_OS_X', 'March_31', 'May_21', 'May_28', 'Metadata', 'Microsoft', 'Microsoft_Windows', 'Operating_system', 'Programming', 'Proprietary_software', 'September_30', 'September_7', 'Software', 'Software_developer', 'Software_license', 'Software_release_life_cycl

Make sure your code works on the Wikipedia graph.  In our implementation, it takes about 4 seconds to generate this matrix from the Wikipedia graph (not including the time of the `read_graph` function).

## Q4: PageRank algorithm

Finally, implement the PageRank algorithm using the adjacency matrix representation. You should use the approach described in the ["Power method" section on the Wikipedia entry](https://en.wikipedia.org/wiki/PageRank#Power_method), which we also discussed in class.

This involves forming some initial uniform probability vector $x$, and repeatly multiplying it by the matrices:
\begin{equation}
x \gets \left(d P + \left(1-d\right)\frac{1}{n} E \right)x
\end{equation}
where $P$ is a transition matrix, $E$ is the matrix of all ones, and $d$ is the damping factor, and $n$ is the number of nodes. You get $P$ by normalizing $A$ so that all columns have sum 1.

*Note*: Here we are defining $d$ to be the "damping factor", which means we will randomly restart with probability $(1-d)$.

Recall that from the definition of PageRank, when we reach a "sink" node (a node with no outgoing edges), we randomly hop to any other node in the network, so that columns of $P$ that have no outgoing edges are set to the uniform distribution.  To be efficient, you'll also want to avoid explicitly forming the $E$ matrix, and should instead use the fact that $E = \mathbf{1}\mathbf{1}^T$ where $\mathbf{1}$ denotes a vector of all ones.  Use the fact that we can reorder matrix multiplication if associative (i.e., the fact the $A(BC)$ = $(AB)C$) to make this operation as fast as possible.

Your function should return a dictionary of nodes and their corresponding page rank.  For example, in the five-node graph from our test cases, we have the following results:
```
C: 0.324
B: 0.281
E: 0.188
D: 0.121
A: 0.085
```
As is intuitive, nodes B and C have higher page rank, as they are pointed to by more of the other nodes.

In [8]:
def pagerank(self, d, iters):
        """Compute the PageRank score for each node in the network.

        Args:
            d: PageRank damping factor d (restart with probability (1-d))
            iters: number of iterations to run

        Returns:
            dict ranks: a dictionary of node: importance score, for each node in the
                network (larger score means higher rank)
        """
        # Get the adjacency matrix and nodes
        adj_matrix, nodes = self.adjacency_matrix()

        # Normalize adjacency matrix to get the transition matrix P
        P = adj_matrix / adj_matrix.sum(axis=0)

        n = len(nodes)

        x = np.ones(n) / n

        # Power iteration method for PageRank
        for _ in range(iters):
            x = d * P.dot(x) + (1 - d) / n

        
        ranks = {nodes[i]: x[i] for i in range(n)}
        sorted_ranks = sorted(ranks.items(), key=lambda x: x[1], reverse=True)

        return dict(sorted_ranks)


# Example usage:

graph = Graph()

edges_list = read_graph()
graph.add_edges(edges_list)


damping_factor = 0.85
iterations = 10


page_ranks = graph.pagerank(damping_factor, iterations)

# Print or use the PageRank scores as needed
print("PageRank Scores:")
for node, score in page_ranks.items():
    print(f"{node}: {score:.2e}")

PageRank Scores:
2007: 1.54e-03
2008: 1.48e-03
United_States: 1.48e-03
Wikimedia_Commons: 1.15e-03
2006: 1.06e-03
France: 9.83e-04
Geographic_coordinate_system: 8.83e-04
United_Kingdom: 8.80e-04
2005: 7.55e-04
Canada: 6.22e-04
World_War_II: 6.14e-04
Germany: 6.08e-04
England: 5.94e-04
Europe: 5.56e-04
2004: 5.30e-04
Japan: 5.19e-04
Population_density: 5.17e-04
Australia: 4.84e-04
London: 4.78e-04
Italy: 4.70e-04
Area: 4.47e-04
India: 4.37e-04
2003: 4.23e-04
New_York_City: 4.20e-04
Spain: 4.16e-04
English_language: 4.04e-04
Russia: 3.80e-04
2002: 3.72e-04
Digital_object_identifier: 3.54e-04
Square_mile: 3.51e-04
2001: 3.50e-04
Departments_of_France: 3.49e-04
January_31: 3.46e-04
Wiktionary: 3.40e-04
China: 3.40e-04
Time_zone: 3.39e-04
World_War_I: 3.32e-04
California: 3.25e-04
2000: 3.17e-04
List_of_countries: 3.17e-04
Sweden: 3.08e-04
Poland: 3.03e-04
Elevation: 3.02e-04
1999: 3.01e-04
North_America: 3.01e-04
January_1: 3.00e-04
New_York: 2.96e-04
Latin: 2.96e-04
Population: 2.96e-04
S

Make sure your implementation works on the full Wikipedia graph (in our implementation, it takes 11 seconds to run, most of which is taken up by generating the adjacency matrix).  The top PageRank entires we get from our implementation are:

```
United_States     2.75e-3
2007              2.44e-3
2008              2.17e-3
Wikimedia_Commons 1.72e-3
United_Kingdom    1.59e-3
2006              1.54e-3
France            1.44e-3
Wiktionary        1.26e-3
Canada            1.09e-3
World_War_II      1.04e-3
2005              1.04e-3
List_of_Africa... 1.00e-3
Germany           0.95e-3
Europe            0.93e-3
English_language  0.90e-3
Geographic_coo... 0.89e-3
Latin             0.88e-3
Australia         0.87e-3
India             0.78e-3
Japan             0.78e-3
```

countries and years! A seemingly reasonable list of pages we may expect to be important.